In [1]:
using Pkg; Pkg.activate(dirname(pwd()))

  Activating project at `/media/yuehhua/Workbench/workspace/machine-learning.jl`


# Ridge regression

## Using MLJ

In [2]:
using MLJ, RDatasets

### Load data

In [3]:
boston = RDatasets.dataset("MASS", "Boston")
first(boston, 6)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222


### Casting scientific types

In [4]:
y, X = unpack(boston, ==(:MedV), colname -> true);
first(X, 6)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222


In [5]:
first(X, 6) |> pretty

┌────────────┬────────────┬────────────┬───────┬────────────┬────────────┬────────────┬────────────┬───────┬───────┬────────────┬────────────┬────────────┐
│ Crim       │ Zn         │ Indus      │ Chas  │ NOx        │ Rm         │ Age        │ Dis        │ Rad   │ Tax   │ PTRatio    │ Black      │ LStat      │
│ Float64    │ Float64    │ Float64    │ Int64 │ Float64    │ Float64    │ Float64    │ Float64    │ Int64 │ Int64 │ Float64    │ Float64    │ Float64    │
│ Continuous │ Continuous │ Continuous │ Count │ Continuous │ Continuous │ Continuous │ Continuous │ Count │ Count │ Continuous │ Continuous │ Continuous │
├────────────┼────────────┼────────────┼───────┼────────────┼────────────┼────────────┼────────────┼───────┼───────┼────────────┼────────────┼────────────┤
│ 0.00632    │ 18.0       │ 2.31       │ 0.0   │ 0.538      │ 6.575      │ 65.2       │ 4.09       │ 1.0   │ 296.0 │ 15.3       │ 396.9      │ 4.98       │
│ 0.02731    │ 0.0        │ 7.07       │ 0.0   │ 0.469      │ 6.

In [6]:
X = coerce(X, autotype(X, rules=(:discrete_to_continuous,)))
first(X, 6)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.09,1.0
2,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0
3,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0
4,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0
5,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0
6,0.02985,0.0,2.18,0.0,0.458,6.43,58.7,6.0622,3.0


### Training/testing set

In [7]:
train, test = partition(eachindex(y), 0.7, shuffle=true)

([175, 246, 486, 263, 15, 258, 318, 320, 425, 380  …  121, 365, 273, 271, 348, 470, 68, 22, 138, 1], [183, 492, 88, 367, 308, 5, 110, 506, 383, 108  …  24, 7, 212, 139, 217, 238, 116, 260, 269, 311])

### Model

In [8]:
RidgeRegressor = @load RidgeRegressor pkg=MLJLinearModels

import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/yuehhua/.julia/packages/MLJModels/lDzCR/src/loading.jl:168


MLJLinearModels.RidgeRegressor

In [9]:
ridgereg = machine(RidgeRegressor(), X, y)

Machine trained 0 times; caches data
  model: RidgeRegressor(lambda = 1.0, …)
  args: 
    1:	Source @549 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @664 ⏎ `AbstractVector{Continuous}`


### Training

In [10]:
fit!(ridgereg, rows=train)

┌ Info: Training machine(RidgeRegressor(lambda = 1.0, …), …).
└ @ MLJBase /home/yuehhua/.julia/packages/MLJBase/rQDaq/src/machines.jl:487
┌ Info: Solver: MLJLinearModels.Analytical
│   iterative: Bool false
│   max_inner: Int64 200
└ @ MLJLinearModels /home/yuehhua/.julia/packages/MLJLinearModels/2qDvV/src/mlj/interface.jl:39


Machine trained 1 time; caches data
  model: RidgeRegressor(lambda = 1.0, …)
  args: 
    1:	Source @549 ⏎ `Table{AbstractVector{Continuous}}`
    2:	Source @664 ⏎ `AbstractVector{Continuous}`


### Predict

In [11]:
ŷ = MLJ.predict(ridgereg, rows=test)

152-element Vector{Float64}:
 32.564351153424795
 10.739517982726504
 25.95424222554394
 19.646946837183215
 32.26054988021896
 27.45963234236864
 18.413702765045187
 24.477172272377643
 13.607703227441004
 19.646952641000592
 16.089338216755962
 22.22454420218037
 25.816156730934573
  ⋮
 23.091942653329294
 15.515047970397923
 14.740754991079058
 24.405463628608956
 14.58677283141349
 13.236585161892123
 24.823835011993914
 31.005882008131298
 19.54893454964359
 35.83977327754937
 37.827154143213846
 21.419565703002277

### Evaluation

In [12]:
rms(ŷ, y[test])

5.477117443114914

### View model parameters

In [13]:
coefs, intercept = fitted_params(ridgereg)
coefs

13-element Vector{Pair{Symbol, Float64}}:
    :Crim => -0.10599459347220104
      :Zn => 0.06807170632729831
   :Indus => 0.010175749393151636
    :Chas => 0.2848165203866192
     :NOx => -0.026374631544308445
      :Rm => 1.0117705419718082
     :Age => 0.0114446180761977
     :Dis => -0.8910995233821151
     :Rad => 0.35711981838190043
     :Tax => -0.020819544570725622
 :PTRatio => -0.7984156060862957
   :Black => 0.006614428900809743
   :LStat => -0.7108393463134475